In [1]:
import numpy as np
from colorama import Fore, Back
import time
from IPython.display import clear_output


def read_file(path):
    with open(path, "r") as f:
        res = f.readlines()

    res = [list(x.strip()) for x in res]
    return np.matrix(res)


input = read_file("example")
input

matrix([['.', '.', '.', '.', '#', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '#'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '#', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '#', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '#', '.', '.', '^', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '.', '.', '#', '.'],
        ['#', '.', '.', '.', '.', '.', '.', '.', '.', '.'],
        ['.', '.', '.', '.', '.', '.', '#', '.', '.', '.']], dtype='<U1')

In [2]:
GUARD_MOVEMENT = {
    "^": [-1, 0],
    ">": [0, 1],
    "v": [1, 0],
    "<": [0, -1],
}

In [3]:
def find_guard(input):
    guards = ["^", ">", "<", "v"]
    coords = np.argwhere(np.isin(input, guards))[0]
    return {"guard": input[tuple(coords)], "coords": coords}


def look_forward(input):
    guard = find_guard(input)
    mod = np.array(GUARD_MOVEMENT[guard["guard"]])
    next_coord = guard["coords"] + mod
    if (
        next_coord[0] > input.shape[0] - 1
        or next_coord[1] > input.shape[1] - 1
        or next_coord[0] < 0
        or next_coord[1] < 0
    ):
        return None
    else:
        return {"value": input[tuple(next_coord)], "coords": next_coord}


def get_next_action(input):
    guard = find_guard(input)
    next_cell = look_forward(input)
    keep_going = True
    if next_cell is None:
        input[tuple(guard["coords"])] = "X"
        keep_going = False
    elif next_cell["value"] == "#":
        keys = list(GUARD_MOVEMENT.keys())
        next_guard_index = (keys.index(guard["guard"]) + 1) % len(keys)
        next_guard = keys[next_guard_index]
        input[tuple(guard["coords"])] = next_guard
    else:
        input[tuple(next_cell["coords"])] = guard["guard"]
        input[tuple(guard["coords"])] = "X"

    return input, keep_going


def print_matrix(input, sleep: float = 1):
    res = input.copy()
    for coord in np.ndindex(res.shape):
        v = input[coord]
        if v == ".":
            res[coord] = "."
        elif v == "#":
            res[coord] = v
        elif v == "X":
            res[coord] = "'"
        else:
            res[coord] = v

    clear_output(wait=True)
    print("\n".join(" ".join(map(str, row)) for row in res.tolist()))
    time.sleep(sleep)

In [5]:
working_input = input.copy()
while True:
    working_input, keep_going = get_next_action(working_input)
    print_matrix(working_input, 0.1)
    if not keep_going:
        break

. . . . # . . . . .
. . . . ' ' ' ' ' #
. . . . ' . . . ' .
. . # . ' . . . ' .
. . ' ' ' ' ' # ' .
. . ' . ' . ' . ' .
. # ' ' ' ' ' ' ' .
. ' ' ' ' ' ' ' # .
# ' ' ' ' ' ' ' . .
. . . . . . # ' . .


In [16]:
def guard_to_tuple(guard):
    return (str(guard["guard"]), str(guard["coords"][0]), str(guard["coords"][1]))


def find_guard(input):
    guards = ["^", ">", "<", "v"]
    coords = np.argwhere(np.isin(input, guards))[0]
    return {"guard": input[tuple(coords)], "coords": coords}


def look_forward(input):
    guard = find_guard(input)
    mod = np.array(GUARD_MOVEMENT[guard["guard"]])
    next_coord = guard["coords"] + mod
    if (
        next_coord[0] > input.shape[0] - 1
        or next_coord[1] > input.shape[1] - 1
        or next_coord[0] < 0
        or next_coord[1] < 0
    ):
        return None
    else:
        return {"value": input[tuple(next_coord)], "coords": next_coord}


def turn_guard(guard):
    guard = dict(guard)
    keys = list(GUARD_MOVEMENT.keys())
    next_guard_index = (keys.index(guard["guard"]) + 1) % len(keys)
    guard["guard"] = keys[next_guard_index]
    guard["coords"] = guard["coords"] + np.array(GUARD_MOVEMENT[guard["guard"]])
    return guard


def get_next_action(input, visited, obs_count):
    guard = find_guard(input)
    next_cell = look_forward(input)
    keep_going = True
    if next_cell is None:
        keep_going = False
    elif next_cell["value"] == "#":
        visited.add(guard_to_tuple(guard))
        keys = list(GUARD_MOVEMENT.keys())
        next_guard_index = (keys.index(guard["guard"]) + 1) % len(keys)
        next_guard = keys[next_guard_index]
        input[tuple(guard["coords"])] = next_guard
    else:
        visited.add(guard_to_tuple(guard))
        ghost_guard = turn_guard(guard)
        if guard_to_tuple(ghost_guard) in visited:
            print(next_cell["coords"])
            obs_count += 1

        input[tuple(next_cell["coords"])] = guard["guard"]
        input[tuple(guard["coords"])] = "X"

    print(obs_count)
    return input, keep_going, obs_count, visited


def print_matrix(input, sleep: float = 1):
    res = input.copy()
    for coord in np.ndindex(res.shape):
        v = input[coord]
        if v == ".":
            res[coord] = "."
        elif v == "#":
            res[coord] = v
        elif v == "X":
            res[coord] = "*"
        else:
            res[coord] = v

    # clear_output(wait=True)
    print("\n".join(" ".join(map(str, row)) for row in res.tolist()))
    time.sleep(sleep)

In [17]:
working_input = input.copy()
visited = set()
obs_count = 0
while True:
    working_input, keep_going, obs_count, visited = get_next_action(
        working_input, visited, obs_count
    )
    print_matrix(working_input, 0.1)
    if not keep_going:
        break

0
. . . . # . . . . .
. . . . . . . . . #
. . . . . . . . . .
. . # . . . . . . .
. . . . . . . # . .
. . . . ^ . . . . .
. # . . * . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . . # . . .
0
. . . . # . . . . .
. . . . . . . . . #
. . . . . . . . . .
. . # . . . . . . .
. . . . ^ . . # . .
. . . . * . . . . .
. # . . * . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . . # . . .
0
. . . . # . . . . .
. . . . . . . . . #
. . . . . . . . . .
. . # . ^ . . . . .
. . . . * . . # . .
. . . . * . . . . .
. # . . * . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . . # . . .
0
. . . . # . . . . .
. . . . . . . . . #
. . . . ^ . . . . .
. . # . * . . . . .
. . . . * . . # . .
. . . . * . . . . .
. # . . * . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . . # . . .
0
. . . . # . . . . .
. . . . ^ . . . . #
. . . . * . . . . .
. . # . * . . . . .
. . . . * . . # . .
. . . . * . . . . .
. # . . * . . . . .
. . . . . . . . # .
# . . . . . . . . .
. . . . . 

In [ ]:
a = set()
g1 = {"guard": ">", "coords": np.array([3, 5])}
g2 = {"guard": "v", "coords": np.array([3, 5])}
a.add(guard_to_tuple(g1))
a.add(guard_to_tuple(g2))

In [10]:
obs_count

4